In [1]:
##imports 

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer


In [2]:
##Needed CSV's
trainingData = pd.read_csv("train-data.csv", index_col=0) 
testingData  = pd.read_csv("test.csv", index_col=0)  

In [3]:
##batch size (started with a small no. to train the algorithm on)
batch_size = 320000

In [4]:
##vectorizing the data
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(trainingData[:batch_size]['OCR text'])

In [5]:
y = trainingData[:batch_size]["Label"]

In [ ]:
print(X)

In [ ]:
print(vectorizer.get_feature_names())

In [ ]:
print(X.shape)

In [6]:
##Creating pipelines for SVM

from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
svm_pipeline = Pipeline([
    ('svm', SGDClassifier(max_iter=320000))
])

In [ ]:
##Listing the testing tuning parameters

import numpy as np
from sklearn.model_selection import GridSearchCV
grid_params = {    
    "svm__loss" : ["hinge", "log", "squared_hinge", "modified_huber"],
    "svm__alpha" : [0.0001, 0.001, 0.01, 0.1],
    "svm__penalty" : ["l2", "l1", "none"],
}
clf = GridSearchCV(svm_pipeline, grid_params,verbose=10)
clf.fit(X, y)
print("Best Score: ", clf.best_score_)
print("Best Params: ", clf.best_params_)


Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2 .............
[CV]  svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2, score=0.723, total=  28.9s
[CV] svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2 .............
[CV]  svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2, score=0.723, total=  30.0s
[CV] svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2 .............
[CV]  svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2, score=0.726, total=  30.8s
[CV] svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2 .............
[CV]  svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2, score=0.725, total=  29.5s
[CV] svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2 .............
[CV]  svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l2, score=0.726, total=  29.4s
[CV] svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l1 .............
[CV]  svm__alpha=0.0001, svm__loss=hinge, svm__penalty=l1, score=0.556, total=  59

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   28.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   58.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  2.0min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:  2.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:  3.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:  4.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  5.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  6.3min remaining:    0.0s


In [18]:
##Fiting the data and training it - modify tuning parameters here based on the GridsearchCV
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(alpha=0.0001, loss='hinge', penalty='none')

clf.fit(X, y)

SGDClassifier(penalty='none')

In [ ]:
##Testing data based on the fitted model

from tqdm import tqdm

#Prediction and accuracy
from sklearn.metrics import accuracy_score
documents1 = testingData[:]["OCR text"]
y_true = testingData[:]['Label']
input = vectorizer.transform(documents1)
prediction1= clf.predict(input)

dataframe = pd.DataFrame(list(y_true),prediction1)
index = 0
count = 0
for i in tqdm(list(y_true)):
    if i == prediction1[index]:
        count += 1
    index+=1
    
    
print("Accuracy:",accuracy_score(list(y_true),prediction1))
print(dataframe)